Impostiamo il path della cartella contenente i file originali e il path della cartella in cui si vuole che vengano memorizzati i file .csv

In [9]:
root=r'C:\Users\Luca\Downloads\acousticbrainz-lowlevel-json-20150129\acousticbrainz-lowlevel-json-20150129\lowlevel'
## inserite in root l'url della cartella in cui avete salvato le cartelle del dataset che hanno nomi ad una sola cifra (0,1,..)

folder=r'C:\Users\Luca\Desktop\Univ\Univ\3 anno\Big Data\PROGETTO_BIGDATA_2019\CSV\caricare'
## inserite in folder l'url della cartella in cui volete che vengano salvati i fle CSV




### chiaramente nell'url che inserite in root devono esserci solo le cartelle che volete elaborare perchè lo script elabora 
### in automatico tutte le cartelle che sono presenti in root

###una volta inseriti root e folder far partire le celle qui sotto e verrà fatto tutto in automatico...
### se potete andate in impostazioni del pc e impostate SOSPENSIONE: MAI....su windows 10:
###  impostazioni -> sistema -> alimentazione e sospensione

Installiamo il modulo 'musicbrainzngs' che ci serve per interrogare il database 'MusicBrainz per recuperare eventualmente i generi

In [ ]:
### questa cella va eseguita solo da Francesca che non ha (credo) ancora installato il modulo 'musicbrainzngs'
!pip install musicbrainzngs

Importiamo i moduli necessari 

In [1]:
import pandas as pd
import json
import musicbrainzngs as mb
import numpy as np
import os

La seguente funzione è necessaria per normalizzare i file originali dei brani in un'unica riga, in quanto restituisce le posizioni dei dati annidati che sono aggregati in liste, i quali andranno trsformati in dati numerici saparati 

In [2]:
def klist(f,l=[]):
    li=[]
    if type(f) is list:
        if type(f[0]) is list:
            return 'll'
        return 'l'
    elif type(f) is dict:
        for k,v in f.items():
            t=klist(v,l+[k])
            if t=='l':
                li.append(l+[k])
            elif t=='ll':
                li.append(l+[k,'l'])  
            elif type(t) is list:
                for i in t:
                    li.append(l+i)
        return li
    else:
        return 'altro'

La seguente funzione prende in input il vettore dei generi originali e restituisce in output il vettore dei generi aggregati.

In [3]:
def assegna_genere(generi_originali):
    l=[]
    for genere_originale in generi_originali:
        g=genere_originale.replace(';',' ').replace('.',' ').replace(',',' ').replace(':',' ').replace('!',' ')\
                         .replace('-',' ').replace('\\',' ').replace('/',' ').lower().split()
        i=0
        done=False
        while i<len(g) and not done:
            
            if 'rock' in g[i] or 'country' in g[i] or\
                ('new' in g[i] and (i+1)<len(g) and 'wave' in g[i+1])\
                or ('punk' in g[i] and not 'hardcore' in g):
                l.append('Rock')
                done=True
                
            elif 'jazz' in g[i]:
                l.append('Jazz')
                done=True
                
            elif 'blues' in g[i]:
                l.append('Blues')
                done=True
                
            elif 'metal' in g[i] or ('hardcore' in g[i] and (i+1)<len(g) and 'punk' in g[i+1])\
                or 'grindcore' in g[i]:
                l.append('Metal')
                done=True
                
            elif 'ambient' in g[i]:
                l.append('Ambient')
                done=True
                
            elif 'funk' in g[i] or ('disco' in g[i] and not 'house' in g and not 'techno' in g):
                l.append('Funk')
                done=True
                
            elif 'rap' in g[i] or ('hip' in g[i] and (i+1)<len(g) and 'hop' in g[i+1]):
                l.append('Hip-Hop')
                done=True
                
            elif 'classic' in g[i] and not ((i+1)<len(g) and 'rock' in g[i+1]):
                l.append('Classical')
                done=True
                
            elif 'house' in g[i] or 'techno' in g[i] or 'hardcore' in g[i] or\
                ('electr' in g[i] and not 'guitar' in g and not 'funk' in g):                                                                                    
                l.append('Techno-House')
                done=True
                
            i+=1
        
        if not done:
            l.append('Pop-Altro')
    return pd.DataFrame({'GENERE':l})

Il codice presente nella seguente cella è il vero e proprio ciclo che trasforma i dati originali in file csv; per la trasformazione dell'intera cartella 'low-level' scaricabile a https://acousticbrainz.org/download sono necessarie almeno 48 ore ininterrotte di elaborazione con un pc di potenza media.

In [7]:
D=pd.DataFrame()  ## inizializziamo il dataframe in cui metteremo i dati

rec_errlist=[]     ### alcune liste che, alla fine del preprocessamento, conterranno il nome dei files per cui è sorto 
art_errlist=[]     ### qualche problema e non è stato possibile recuperare il genere. Questi file non vengono inseriti nel 
notaglist=[]       ### data-frame D e si dovrà decidere che farne
structerr=[]


for fold in os.listdir(root):
    
    sbfold_list=os.listdir(root+'\\'+fold)
    
    for sbfold in sbfold_list:
        
        files=os.listdir(root+'\\'+fold+'\\'+sbfold)
        
        del D
        D=pd.DataFrame()
    
        for file in files:   

            with open(root+'\\'+fold+'\\'+sbfold+'\\'+file,'r') as f:   
                jfile=json.load(f)

            ## gen indica se il genere verrà preso direttamente dal file o se verrà effettuata un'interrogazione        
            ## infatti alcuni dei file hanno il genere nella sezione metadata, altri no

            gen='file'    



            ## se la chiave 'genere' non è presente nel dizionario si  interrogherà il database per ottenerlo
            try:
                jfile['genere']=jfile['metadata']['tags']['genre'][0]   
            except:                                                     
                gen='int'



            ## con il prossimo if si controlla che il genere, anche se presente come chiave, non sia una stringa vuota o una stringa 
            ##di spazi

            if gen=='file' and len(jfile['genere'].replace(' ',''))==0:    
                gen='int'                                                  



            ## qui inzia il blocco eseguito solo se è necessario ottenere il genere con un interrogazone del database
            if gen=='int':

                ##alcuni files non hanno l'etichetta 'musicbrainz_recordingid' in ['metadata']['tags'] perciò
                ## si usa try except in caso ci sia un KeyError
                try:
                    jfile['rec_id']=jfile['metadata']['tags']['musicbrainz_recordingid'][0]  
                except:

                    ## se si presenta un errore si prova comunque ad accedere a 'musicbrainz_artistid'; se c'è di nuovo errore
                    ## il genere non è recuperabile in nessun modo e si passa al file successivo con continue
                    try:
                        jfile['artist_id']=jfile['metadata']['tags']['musicbrainz_artistid'][0]
                    except:
                        continue

                inter='r'     ### variabile che ci dice se i risultati sono presi dal campo 'recording' o dal campo 'artist'


                ## si prova ad interrogare il database tramite il recording_id; se c'è un errore (perchè nel punto precedente
                ## non si è potuto registrare il recording_id ma solo l'artist_id oppure perchè c'è un errore di connessione)
                ## si prova interrogando tramite artist_id
                try:
                    results=mb.get_recording_by_id(jfile['rec_id'],includes=['tags','user-tags'])
                except:
                    rec_errlist.append(root+fold+sbfold+file)
                    ## se c'è stato errore si prova interroganto tramite artist_id; se si ha successo
                    ## la variabile inter assume valore 'a'. Se c'è di nuovo errore il genere non è
                    ## in alcun modo recuperabile e si passa al file successivo
                    try:
                        results=mb.get_artist_by_id(jfile['artist_id'],includes=['tags','user-tags'])
                        inter='a'
                    except:
                        art_errlist.append(root+fold+sbfold+file)
                        continue

                ## di seguito i due blocchi per il recupero del genere dai risultati dell'interrogazione; i due blocchi if 
                ## sono necessari perchè a seconda che l'interrogazione sia stata fatta al campo artist o recording, le chiavi 
                ## per ottenere i tag sono diverse

                ## in entrambi i blocchi si usa comunque un costrutto try except perchè in alcuni casi non è presente la chiave
                ## 'tag-list' nei risultati; in tal caso e necessario gestire l'errore
                if inter=='r':
                    try:
                        maxcount=max(tag['count'] for tag in results['recording']['tag-list'])
                    except:
                        try:
                            results=mb.get_artist_by_id(jfile['artist_id'],includes=['tags','user-tags'])
                            inter='a'
                        except:
                            art_errlist.append(root+fold+sbfold+file)
                            continue
                if inter=='a':
                    try:
                        maxcount=max(tag['count'] for tag in results['artist']['tag-list'])
                    except:
                        notaglist.append(root+fold+sbfold+file)
                        continue


                if inter=='r':
                    jfile['genere']=[tag['name'] for tag in results['recording']['tag-list'] if tag['count']==maxcount][0]
                else:
                    jfile['genere']=[tag['name'] for tag in results['artist']['tag-list'] if tag['count']==maxcount][0]


                try:
                    del jfile['rec_id']
                except:
                    pass
                try:
                    del jfile['artist_id']
                except:
                    pass

            try:
                del jfile['rhythm']['beats_position']
            except:
                pass
            try:
                del jfile['metadata']
            except:
                pass

            vardiag_inv=np.diag(np.diag(jfile['lowlevel']['gfcc']['cov'])**(-0.5))
            jfile['lowlevel']['gfcc']['cov']=vardiag_inv.dot(np.array(jfile['lowlevel']['gfcc']['cov']).dot(vardiag_inv))
            jfile['lowlevel']['gfcc']['cov']=np.linalg.det(jfile['lowlevel']['gfcc']['cov'])

            vardiag_inv=np.diag(np.diag(jfile['lowlevel']['mfcc']['cov'])**(-0.5))
            jfile['lowlevel']['mfcc']['cov']=vardiag_inv.dot(np.array(jfile['lowlevel']['mfcc']['cov']).dot(vardiag_inv))
            jfile['lowlevel']['mfcc']['cov']=np.linalg.det(jfile['lowlevel']['mfcc']['cov'])

            del jfile['lowlevel']['gfcc']['icov']
            del jfile['lowlevel']['mfcc']['icov']  

            t=klist(jfile)

            for i in t:
                if len(i)==2:
                    jfile[i[0]][i[1]]={str(k):v for k,v in enumerate(jfile[i[0]][i[1]])}
                else:
                    jfile[i[1]][i[2]][i[3]]={str(k):v for k,v in enumerate(jfile[i[1]][i[2]][i[3]])} 

            if len(klist(jfile))!=0:
                structerr.append(root+fold+sbfold+file)
                print('errore di struttura')
                continue

            D=pd.concat([D,pd.io.json.json_normalize(jfile)],ignore_index=True) 
            del jfile
        
        
        with open(folder+'\\'+fold+'_'+sbfold+'.csv','w'):
            pass
        
        pd.concat([assegna_genere(D['genere']),D],axis=1).to_csv(folder+'\\'+fold+'_'+sbfold+'.csv',index=False)

C:\Users\Luca\Anaconda3\envs\msbd\lib\site-packages\ipykernel_launcher.py:156: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Luca\\Desktop\\Univ\\Univ\\3° Anno\\Big Data\\PROGETTO_BIGDATA_2019\\CSV\\4_46.csv'